In [1]:
import torch
import torch.nn as nn

from PIL import Image
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
from torchvision import transforms

In [2]:
TEST_SIZE=0.2
IMG_SIZE = 64
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD = [0.229, 0.224, 0.225]
DATASET_NAME = "cats_vs_dogs"
datasets = load_dataset(DATASET_NAME)
datasets
datasets = datasets["train"].train_test_split(test_size=TEST_SIZE)
img_transforms = transforms.Compose(
        [
                transforms.Resize((IMG_SIZE,IMG_SIZE)),
                transforms.Grayscale(),
                transforms.ToTensor(),
                transforms.Normalize(
                        NORMALIZE_MEAN,
                        NORMALIZE_STD
                )
        ]
)

Generating train split:   0%|          | 0/23410 [00:00<?, ? examples/s]

In [3]:
class CatDogDataset(Dataset):
        def __init__(self, data, transform=None):
                self.data = data
                self.transform = transform

        def __len__(self):
                return len(self.data)
        
        def __getitem__(self, idx):
                images = self.data[idx]["image"]
                labels = self.data[idx]["labels"]

                if self.transform:
                        images = self.transform(images)
                
                labels =torch.tensor(labels, dtype=torch.long)
                return images, labels

In [4]:
TRAIN_BATCH_SIZE = 512
VAL_BATCH_SIZE = 256

train_dataset = CatDogDataset(
        datasets["train"], transform=img_transforms
)

test_dataset = CatDogDataset(
        datasets["test"], transform=img_transforms
)

train_loader = DataLoader(
        train_dataset,
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=True
)

test_loader = DataLoader(
        test_dataset,
        batch_size=VAL_BATCH_SIZE,
        shuffle=False
)

In [ ]:
class CatDogModel(nn.Module):
        def __init__(self, n_classes):
                super(CatDogModel, self).__init__()
        
                resnet_model = resnet18(weights="IMAGENET1K_V1")
                self.backbone = nn.Sequential(*list(resnet_model.children())[:-1])
                for param in self.backbone.parameters():
                        param.requires_grad = False

                in_features = resnet_model.fc.in_features
                self.fc = nn.Linear(in_features, n_classes)

        def forward(self, x):
                x = self.backbone(x)
                x = torch.flatten(x)
                x = self.fc(x)
                return x

In [5]:
device  = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
N_CLASSES = 2
model = CatDogModel(N_CLASSES).to(device)
test_input = torch.rand(1,3, 224, 224).to(device)
with torch.no_grad():
        output = model(test_input)
        print(output.shape)

In [ ]:
EPOCHS = 3
LR = 1e-3
WEIGHT_DECAY = 1e-5

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
losses= torch.nn.CrossEntropyLoss()

for  epoch in range(EPOCHS):
        train_losses = []
        model.train()
        for images, labels in train_loader:
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)

                optimizer.zero_grad()
                loss = losses(outputs, labels)
                loss.backward()
                optimizer.step()

                train_losses.append(loss.item())
        
        train_loss = sum(train_losses) / len(train_losses)

        val_losses = []
        model.eval()
        with torch.no_grad():
                for images, labels in test_loader:
                        images = images.to(device)
                        labels = labels.to(device)

                        outputs = model(images)
                        loss = losses(outputs, labels)
                        val_losses.append(loss.item())

        val_losses = sum(val_losses) / len(val_losses)
        print(f"EPOCH  {epoch+1}: \tTrain Loss: {train_losses:.3f}\tValidation Loss: {val_losses:.3f}")

In [ ]:
SAVE_PATH = "catdog_weights.pt"
torch.save(model.state_dict(), save_path=SAVE_PATH)